In [1]:
import os
import sys
import requests
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#print(os.path.realpath(__file__))
#print(os.path.dirname(os.path.realpath(__file__)))
print(os.path.realpath('..'))

sys.path.append('..')
from src.utils.functions import import_data

/home/daniel/Proyectos_Daniel/finapp


Different ways to access Dir
- `os.path.realpath(__file__)`
- `os.path.abspath('')`
- `os.path.dirname(os.path.realpath(__file__))`
- `os.path.realpath('./')`

In [2]:
TOKEN = os.environ.get('NOTION_TOKEN')
DATABASE_ID = os.environ.get('DATABASE_ID')

In [3]:
# import the data to a dataframe
data = import_data(TOKEN, DATABASE_ID)
data

200


,Date,Name,Category,Amount,Pay_Method
0,2024-03-22,Perritos,Fast food,15000.0,T_Débito
1,2024-03-22,New Expense,,NaN,None
2,2024-03-22,Latte frío,Fast food,6900.0,T_Débito
3,2024-03-21,Pan sandwiches,Food,5500.0,Efectivo
4,2024-03-21,Recarga civica,Transport,10000.0,T_Débito
...,...,...,...,...,...
68,2024-03-01,Café Juan V,Fast food,38500.0,Rappi
69,2024-03-01,Burritos,Fast food,14000.0,T_Débito
70,2024-03-01,Zapatos Papá,Shopping,90000.0,T_Débito
71,2024-03-01,Dinero Tío Miller,Extra,100000.0,T_Débito


In [4]:
df = data.dropna()
df

,Date,Name,Category,Amount,Pay_Method
0,2024-03-22,Perritos,Fast food,15000.0,T_Débito
2,2024-03-22,Latte frío,Fast food,6900.0,T_Débito
3,2024-03-21,Pan sandwiches,Food,5500.0,Efectivo
4,2024-03-21,Recarga civica,Transport,10000.0,T_Débito
5,2024-03-20,Comida Procinal,Entertainments,48900.0,Rappi
...,...,...,...,...,...
68,2024-03-01,Café Juan V,Fast food,38500.0,Rappi
69,2024-03-01,Burritos,Fast food,14000.0,T_Débito
70,2024-03-01,Zapatos Papá,Shopping,90000.0,T_Débito
71,2024-03-01,Dinero Tío Miller,Extra,100000.0,T_Débito


In [14]:
df_grouped = df.groupby('Category')['Amount'].sum().reset_index()
fig = px.pie(df_grouped, values='Amount', names='Category', title='Montos por Categoría')

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title=dict(font=dict(size=20)), 
                  
                #paper_bgcolor='rgba(0,0,0,0)', 
                #plot_bgcolor='rgba(0,0,0,0)'
                  )
fig.update_xaxes(
    showline=True,mirror=True,linecolor='black',
    nticks=10,ticks="inside", 
    title_font = {"size": 15},
    showgrid=True,range=[0,22],
    title_standoff = 20
    )

fig.update_yaxes(
    showline=True,mirror=True,linecolor='black',
    nticks=10,ticks="inside",tickprefix="$",
    title_font = {"size": 15},range=[0,1100],
    title_standoff = 20)

fig.update_layout(showlegend=True,
                  legend=dict(
                      yanchor="top", 
                      y=0.99,
                      xanchor="left",
                      x=0.15,
                      bordercolor="Black",
                      borderwidth=1)
                 )

fig.write_image('/home/daniel/Proyectos_Daniel/finapp/data/images/pie_px.png', format='png')
fig.show()

In [9]:
df_grouped

,Category,Amount
0,Bills & Utilities,2777191.0
1,Entertainments,567000.0
2,Extra,342500.0
3,Fast food,636536.0
4,Food,488733.0
5,Health & Wellness,14300.0
6,Shopping,171712.0
7,Transport,116800.0


In [10]:
df.query('Category == ""')

,Date,Name,Category,Amount,Pay_Method
